# Malaria Cell Detection with Deep Learning

Malaria remains one of the most severe public health challenges worldwide, particularly in sub-Saharan Africa. Caused by the *Plasmodium* parasite and transmitted through the bites of infected Anopheles mosquitoes, malaria leads to hundreds of thousands of deaths annually, with children under five being the most vulnerable.

Early and accurate diagnosis is critical for effective treatment and reducing mortality. Traditional methods, such as microscopic examination of blood smears, are time-consuming, require trained personnel, and are prone to human error.

Automated detection of malaria-infected cells using digital microscopy images and deep learning offers a scalable and fast alternative. Convolutional Neural Networks (CNNs) and transfer learning models can learn to identify parasitized cells from healthy ones by extracting complex features from high-dimensional image data, enabling accurate and rapid diagnosis.

## Key Impacts:

- **Medical & Public Health:** Reduces diagnostic errors, accelerates treatment initiation, and contributes to malaria control programs.
- **Efficiency:** Automates the labor-intensive process of manual blood smear analysis, allowing laboratory staff to focus on critical tasks.
- **Scalability:** Can be deployed in resource-limited settings where trained microscopists are scarce, improving healthcare accessibility.

This project explores the use of classical machine learning models, a custom CNN, and transfer learning with VGG16 to build an automated pipeline for malaria cell detection. The goal is to create a robust, accurate, and generalizable model capable of distinguishing parasitized cells from healthy ones in digital blood smear images.

## Step 1: Data Setup and Download

In this step, we set up the environment, import the necessary libraries, and download the dataset using the **KaggleHub API**.  
The dataset contains images of **Parasitized** and **Uninfected** cells, which will later be used for binary classification.  

We will also verify that the dataset has been downloaded and explore the structure to confirm that both classes exist.

In [ ]:
# Step 1: We started by setting up, downloading and verifying the dataset

# Installation of kagglehub
!pip install -q kagglehub --upgrade

# Imports and reproducibility
import os
from pathlib import Path
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# For this we downloaded the dataset from Kagglehub
import kagglehub

In [ ]:
print("Requesting dataset download... (this may take a minute)")

dataset_path = kagglehub.dataset_download("iarunava/cell-images-for-detecting-malaria")
print("kagglehub returned path:", dataset_path)

# Set base_dir 
base_dir = Path(dataset_path)
if base_dir.is_file() and str(base_dir).lower().endswith('.zip'):
    print("Downloaded a zip file. Extracting...")
    import zipfile
    extract_to = Path("/kaggle/working/cell_images")
    with zipfile.ZipFile(str(base_dir), 'r') as z:
        z.extractall(extract_to)
    base_dir = extract_to
    print("Extracted to:", base_dir)

if (base_dir / "cell_images").exists():
    base_dir = base_dir / "cell_images"

parasitized_dir = base_dir / 'Parasitized'
uninfected_dir = base_dir / 'Uninfected'

# Conducted Safety Checks for the Dataset
assert parasitized_dir.exists(), f"Parasitized folder not found at {parasitized_dir}"
assert uninfected_dir.exists(), f"Uninfected folder not found at {uninfected_dir}"

parasitized_count = len(list(parasitized_dir.glob('*.png')))
uninfected_count = len(list(uninfected_dir.glob('*.png')))

print(f"Parasitized samples: {parasitized_count}")
print(f"Uninfected samples:  {uninfected_count}")
print(f"Base data directory: {base_dir}")

# Printed a few example file names
print("\nExample Parasitized files (first 3):")
for p in list(parasitized_dir.glob('*.png'))[:3]:
    print(" ", p.name)
print("\nExample Uninfected files (first 3):")
for p in list(uninfected_dir.glob('*.png'))[:3]:
    print(" ", p.name)

## Step 2: Data Visualization and Class Distribution

Before building any models, it's essential to visualize the dataset and confirm that both classes are well represented.  
In this step, we will:
- Plot a few random images from each class (*Parasitized* and *Uninfected*).
- Display a simple bar chart showing the number of images in each class.

This helps us understand the dataset and verify that the data was loaded correctly.

In [ ]:
# Step 2: Visualize Images and Class Counts

import random

# Get random samples from each class
parasitized_samples = random.sample(list(parasitized_dir.glob('*.png')), 4)
uninfected_samples = random.sample(list(uninfected_dir.glob('*.png')), 4)

# Display random images from both classes
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
fig.suptitle('Sample Images from Each Class', fontsize=16)

for i, img_path in enumerate(parasitized_samples):
    img = mpimg.imread(img_path)
    axes[0, i].imshow(img)
    axes[0, i].set_title('Parasitized')
    axes[0, i].axis('off')

for i, img_path in enumerate(uninfected_samples):
    img = mpimg.imread(img_path)
    axes[1, i].imshow(img)
    axes[1, i].set_title('Uninfected')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

# Plot class distribution
plt.figure(figsize=(5, 4))
plt.bar(['Parasitized', 'Uninfected'], [parasitized_count, uninfected_count], color=['#e74c3c', '#2ecc71'])
plt.title('Class Distribution')
plt.ylabel('Number of Images')
plt.show()

## Step 3: Data Preprocessing and Splitting

To train our models effectively, we need to preprocess the images and divide the dataset into **training** and **validation** sets.

In this step:
- We'll use `ImageDataGenerator` from Keras to rescale pixel values and apply **data augmentation** (like zooming, shearing, and flipping) to make the model more robust.
- We'll create two generators:
  - **Training Generator:** Applies augmentation and rescaling.
  - **Validation Generator:** Only applies rescaling (no augmentation).
- Finally, we'll verify that the generators are working by printing their structure.

In [ ]:
# Step 3: Data Preprocessing and Generators

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image dimensions and batch size
IMG_HEIGHT = 150
IMG_WIDTH = 150
BATCH_SIZE = 32

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# Validation generator (no augmentation)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Create training and validation generators
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

validation_generator = val_datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Class indices: {train_generator.class_indices}")

## Step 4: Building Deep Learning Models

We'll implement multiple approaches to compare their effectiveness:

1. **Custom CNN**: A convolutional neural network built from scratch
2. **Transfer Learning with VGG16**: Using pre-trained weights from ImageNet

Let's start with building our custom CNN architecture.

In [ ]:
# Step 4: Build Custom CNN Model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def create_custom_cnn():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        MaxPooling2D(2, 2),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        
        Flatten(),
        Dropout(0.5),
        Dense(512, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create and display the custom CNN
custom_model = create_custom_cnn()
custom_model.summary()

In [ ]:
# Transfer Learning with VGG16

from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D

def create_vgg16_model():
    # Load pre-trained VGG16 model
    base_model = VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
    )
    
    # Freeze base model layers
    base_model.trainable = False
    
    # Add custom classification head
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create VGG16 model
vgg16_model = create_vgg16_model()
print(f"VGG16 model created with {vgg16_model.count_params()} parameters")

## Step 5: Model Training

Now we'll train both models and compare their performance. We'll use callbacks for early stopping and learning rate reduction to optimize training.

In [ ]:
# Training setup with callbacks

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

callbacks = [early_stop, reduce_lr]

EPOCHS = 15

In [ ]:
# Train Custom CNN
print("Training Custom CNN...")

history_custom = custom_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

print("Custom CNN training completed!")

In [ ]:
# Train VGG16 Transfer Learning Model
print("Training VGG16 Transfer Learning Model...")

history_vgg16 = vgg16_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

print("VGG16 training completed!")

## Step 6: Model Evaluation and Comparison

Let's evaluate both models and visualize their training progress and performance.

In [ ]:
# Plot training history

def plot_training_history(history, title):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title(f'{title} - Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)
    
    # Plot loss
    ax2.plot(history.history['loss'], label='Training Loss')
    ax2.plot(history.history['val_loss'], label='Validation Loss')
    ax2.set_title(f'{title} - Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

# Plot training histories
plot_training_history(history_custom, 'Custom CNN')
plot_training_history(history_vgg16, 'VGG16 Transfer Learning')

In [ ]:
# Model evaluation

# Evaluate Custom CNN
custom_loss, custom_acc = custom_model.evaluate(validation_generator, verbose=0)
print(f"Custom CNN - Validation Accuracy: {custom_acc:.4f}")
print(f"Custom CNN - Validation Loss: {custom_loss:.4f}")

# Evaluate VGG16
vgg16_loss, vgg16_acc = vgg16_model.evaluate(validation_generator, verbose=0)
print(f"VGG16 - Validation Accuracy: {vgg16_acc:.4f}")
print(f"VGG16 - Validation Loss: {vgg16_loss:.4f}")

# Compare models
print("\n" + "="*50)
print("MODEL COMPARISON SUMMARY")
print("="*50)
print(f"Custom CNN Accuracy: {custom_acc:.4f} ({custom_acc*100:.2f}%)")
print(f"VGG16 Accuracy: {vgg16_acc:.4f} ({vgg16_acc*100:.2f}%)")

if vgg16_acc > custom_acc:
    print(f"\n🏆 Winner: VGG16 Transfer Learning")
    print(f"Performance gain: {((vgg16_acc - custom_acc) * 100):.2f} percentage points")
else:
    print(f"\n🏆 Winner: Custom CNN")
    print(f"Performance gain: {((custom_acc - vgg16_acc) * 100):.2f} percentage points")

## Step 7: Model Predictions and Visualizations

Let's visualize some predictions to understand how well our models are performing on individual samples.

In [ ]:
# Visualize predictions

def visualize_predictions(model, generator, model_name, num_images=8):
    # Get a batch of images
    batch_images, batch_labels = next(generator)
    
    # Make predictions
    predictions = model.predict(batch_images)
    predicted_classes = (predictions > 0.5).astype(int)
    
    # Plot results
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.ravel()
    
    class_names = ['Parasitized', 'Uninfected']
    
    for i in range(min(num_images, len(batch_images))):
        axes[i].imshow(batch_images[i])
        
        true_label = int(batch_labels[i])
        pred_label = predicted_classes[i][0]
        confidence = predictions[i][0]
        
        # Color coding: green for correct, red for incorrect
        color = 'green' if true_label == pred_label else 'red'
        
        title = f'True: {class_names[true_label]}\nPred: {class_names[pred_label]}\nConf: {confidence:.3f}'
        axes[i].set_title(title, color=color, fontsize=10)
        axes[i].axis('off')
    
    plt.suptitle(f'{model_name} - Sample Predictions', fontsize=16)
    plt.tight_layout()
    plt.show()

# Show predictions from both models
visualize_predictions(custom_model, validation_generator, 'Custom CNN')
visualize_predictions(vgg16_model, validation_generator, 'VGG16 Transfer Learning')

## Conclusion and Future Work

### Key Achievements:

✅ **Successfully implemented** deep learning models for malaria cell detection

✅ **Compared multiple approaches** - Custom CNN vs Transfer Learning with VGG16

✅ **Applied proper data preprocessing** with augmentation techniques

✅ **Achieved high accuracy** in distinguishing parasitized from uninfected cells

### Medical Impact:

- **Faster diagnosis** - Automated detection reduces analysis time
- **Reduced human error** - Consistent, objective analysis
- **Scalable solution** - Can be deployed in resource-limited settings
- **Early detection** - Enables prompt treatment initiation

### Future Enhancements:

1. **Real-time detection system** for live microscopy
2. **Mobile application** for field deployment
3. **Multi-species detection** to identify different Plasmodium species
4. **Integration with lab systems** for automated workflows
5. **Clinical validation** studies in healthcare settings

This project demonstrates the potential of AI in healthcare, specifically in making malaria diagnosis more accessible, accurate, and efficient worldwide.